# 项目背景
	  联合国粮食及农业组织最近的一份报告表明，每年农业生产的自然损失中有三分之一以上是由农业病虫害造成的，使这些成为当前影响农业生产和农业生产的最重要因素。需要考虑的农业病虫害众多，依赖于实验室观察和实验的传统方法很容易导致错误的诊断。除此之外，缺乏专业的农业技术人员往往难以及时发现病虫害以采取适当的补救措施。为了克服这些问题，许多研究人员转向使用机器学习方法和计算机视觉技术来识别农业病虫害。这首先涉及分析和处理与植物病虫害相关的图像数据。在此之后，建立机器学习模型以获得与不同图像特征相关的不同层次。最后，使用分类器来快速准确地识别不同类型的病虫害。所有采用这种方法的研究的最终目的都是为农业病虫害的防治提供技术指导。
	  农业病害的图像识别比农业病虫害的识别更具挑战性。各种机器学习方法已经解决了这个问题。这些包括聚类方法、SVM分类器、贝叶斯分类器和浅层神经网络方法。许多这项工作正在进行中。然而，传统的机器学习方法在用于农业病害图像识别的实际应用中，往往存在一些不足：它们高度依赖于原始疾病图像的质量；这些方法的实现通常非常复杂；如果训练样本的数量很大，使用这些传统的机器学习方法很难有效地构建相应的模型。
	  随着现代智能农业的发展，使得使用更先进、更智能的机器学习方法来利用这些数据提供的机会来提高农业病害图像的有效性变得越来越重要。机器学习方法的最新进展，如深度学习和迁移学习，在许多应用领域取得了重大突破，并开始被用于农业病害图像识别。

# 创新点
同国外相比，我国对于农作物病害识别的应用研究任重道远。深度学习在农作物病害智能识别的应用有：

	①耕眼App：实现柑橘、葡萄、芒果、蔬菜病害识别及相关防治方案等

	②识农微信小程序：实现柑橘、葡萄、芒果、苹果、水稻、小麦病虫害识别及相关防治方案等。

但目前的相关深度学习农作物病虫害应用是基于现有深度学习基本框架下的集成应用，主要存在以下不足：

	①在识别精度上有待提升

	②在可识别的农作物种类上有待增加

	③对于病害的粒度有待细分等

综上所述，本作品基于AI Challenger农作物叶子图像数据集包含10种植物的27种病害，合计61个分类(按“物种-病害-程度”分)的特性，主要从以下几个方面考虑改进：

	①增加一定的农作物种类

	②优选模型，注重预测精度

	③训练调优实现精度提升

	④对农作物的病害程度实现一定细化

	⑤开发农作物病害智能识别系统

# 数据来源与数据详情
本研究基于AI Challenger农作物叶子图像数据集包含10种植物(苹果、樱桃、葡萄、柑桔、桃、草莓、番茄、辣椒、玉米、马铃薯)的27种病害（其中24个病害有分一般和严重两种程度），合计61个分类(按“物种-病害-程度”分)的特性，训练图像总数为31718张，测试图像总数为4540张。每张图包含一片农作物的叶子，叶子占据图片主要位置。

![](https://ai-studio-static-online.cdn.bcebos.com/640050cd47a94c59a4d70107a7968db1861bb60e928b4386a82a81eb017dcea3)


# 解压数据集

In [ ]:
!unzip /home/aistudio/data/data101323/data.zip

# 数据读取

In [ ]:

import paddle
import paddle.nn.functional as F
import numpy as np
import cv2
import json
import math
import random
import os
from paddle.io import Dataset  # 导入Datasrt库

filename = "AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations.json"
f_open = open(filename)
fileJson = json.load(f_open)

train_data = []


for i in range(len(fileJson)):
    img1=cv2.imread("AgriculturalDisease_trainingset/images/"+fileJson[i]['image_id'])
    img2=cv2.resize(img1, (128,128), interpolation=cv2.INTER_AREA)/255
    r=[]
    g=[]
    b=[]

    r.append(img2[:, :, 0])
    g.append(img2[:, :, 1])
    b.append(img2[:, :, 2])

    one_data = np.concatenate((r,g,b),axis=0)
    one_data = paddle.to_tensor(one_data,dtype="float32")
    train_data.append([one_data,fileJson[i]['disease_class']])

filename = "AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations.json"
f_open = open(filename)
fileJson1 = json.load(f_open)
test_data = []


for i in range(len(fileJson1)):

    img1=cv2.imread("AgriculturalDisease_validationset/images/"+fileJson1[i]['image_id'])
    img2=cv2.resize(img1, (128,128), interpolation=cv2.INTER_AREA)/255

    r=[]
    g=[]
    b=[]

    r.append(img2[:, :, 0])
    g.append(img2[:, :, 1])
    b.append(img2[:, :, 2])

    one_data = np.concatenate((r,g,b),axis=0)
    one_data = paddle.to_tensor(one_data,dtype="float32")
    test_data.append([one_data,fileJson1[i]['disease_class']])

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


# 网络选择
  	红色的线是 EfficientNet 的曲线，横轴为模型大小，纵轴为准确率。从图中可以看出几乎碾压了众多之前熟悉的经典网络模型。它使用一个简单而高效的复合系数来从depth, width, resolution 三个维度放大网络，不会像传统的方法那样任意缩放网络的维度，基于神经结构搜索技术可以获得最优的一组参数(复合系数)。

![](https://ai-studio-static-online.cdn.bcebos.com/0cf81564e89a4628bb895139c6a9adc94379ac100a134bb8ac284df64ca56abc)


# 网络结构
![](https://ai-studio-static-online.cdn.bcebos.com/c05b1c96f1b447f0aac7fba5e3f3583102a8fecebd534664b018bbafa6ce8061)


# 打印出本项目网络结构


In [3]:
from Net import EfficientNet 
import paddle
model = EfficientNet.from_name('efficientnet-b4')
model = paddle.Model(model)
model.summary((-1, 3, 224, 224))

	具体的模型代码在Net文件中

![](https://ai-studio-static-online.cdn.bcebos.com/f3dd755357bf49aa81ba865c3078d7cef75de833c99d4d40a9c444672bf6c563)


In [ ]:
from Net import EfficientNet 
print("-------end readData--------")
class MyDataset(Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, mode='train'):
        """
        步骤二：实现构造函数，定义数据读取方式，划分训练和测试数据集
        """
        super(MyDataset, self).__init__()

        if mode == 'train':
            self.data = train_data
        else:
            self.data = test_data

    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        data = self.data[index][0]
        label = self.data[index][1]

        return data, label

    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        return len(self.data)

# s_tra_data,s_tra_label = split_data(train_data,train_label,batch_size=32)
# s_tes_data,s_tes_label = split_data(test_data,test_label,batch_size=32)
#数据读取
train_loader = paddle.io.DataLoader(MyDataset("train"), batch_size=16, shuffle=True)
test_loader = paddle.io.DataLoader(MyDataset("test"), batch_size=16, shuffle=True)

epoch_num = 20 #训练轮数
batch_size = 16 
learning_rate = 0.0001 #学习率

val_acc_history = []
val_loss_history = []


def train(model):
    print('start training ... ')
    # turn into training mode
    model.train()

    opt = paddle.optimizer.Adam(learning_rate=learning_rate,
                                parameters=model.parameters())

    for epoch in range(epoch_num):
        acc_train = []
        for batch_id, data in enumerate(train_loader()):
            x_data = data[0]
            y_data = paddle.to_tensor(data[1],dtype="int64")
            y_data = paddle.unsqueeze(y_data, 1)
            logits = model(x_data)
            loss = F.cross_entropy(logits, y_data)
            acc = paddle.metric.accuracy(logits, y_data)
            acc_train.append(acc.numpy())
            if batch_id % 100 == 0:
                print("epoch: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, loss.numpy()))
                avg_acc = np.mean(acc_train)
                print("[train] accuracy: {}".format(avg_acc))
            loss.backward()
            opt.step()
            opt.clear_grad()
        
        # evaluate model after one epoch
        model.eval()
        accuracies = []
        losses = []
        for batch_id, data in enumerate(test_loader()):
            x_data = data[0]
            y_data = paddle.to_tensor(data[1],dtype="int64")
            y_data = paddle.unsqueeze(y_data, 1)

            logits = model(x_data)
            loss = F.cross_entropy(logits, y_data)
            acc = paddle.metric.accuracy(logits, y_data)
            accuracies.append(acc.numpy())
            losses.append(loss.numpy())

        avg_acc, avg_loss = np.mean(accuracies), np.mean(losses)
        print("[test] accuracy/loss: {}/{}".format(avg_acc, avg_loss))
        val_acc_history.append(avg_acc)
        val_loss_history.append(avg_loss)
        model.train()
model = EfficientNet.from_name('efficientnet-b4')

#train(model)  
paddle.save(model.state_dict(), "model.pdparams")#保存模型

# 项目的落地实现

本作品优选出最佳的网络模型以开发 Web 系统下的农作物病害识别系统。本系统由管理员系统、农户系统和专家系统组成。本系统由登录功能、注册功能、农作物病害查询、病害识别等功能构成。进入系统需要注册和登录，分别为普通用户和专家。如图1所示

![](https://ai-studio-static-online.cdn.bcebos.com/17a0e49dbb4741bf9a99cc4e9adbaaf53a8c38bed8074fe1a00499b30faff876)
![](https://ai-studio-static-online.cdn.bcebos.com/a084f4f2ec5446e4a825918326c2c0d3656750e13aa04710af68ee9df44b379d)



# 农户系统

农户进入系统首页，可以查询本系统所有的农作物病害种类及相关信息；也可以通过上传待识别图片进行识别，获取相关病害信息；若有疑问，还可以向专家提出问题，获得解答。如图2所示

![](https://ai-studio-static-online.cdn.bcebos.com/874e84829f5b4bdaa3ce9abec41cf9288f8d8a529c6949d2ba5c1e2c75670160)
![](https://ai-studio-static-online.cdn.bcebos.com/34da9bc7672c443a9ea0b7f55ffefacb08ee83046e284c638cb884e5a4c40322)

其中农作物病害识别流程是：农户选着上传图片——点击图片识别按钮识别图片——存在在疑问可以通过提问咨询专家——识别完成。如图3、4、5所示![](https://ai-studio-static-online.cdn.bcebos.com/72d985764f214e76af49662800adbb52059e6cae7cc24f66b85a3499a2191413)
								
                                	图3 农户选着上传图片

![](https://ai-studio-static-online.cdn.bcebos.com/789505ac7cb04982bff761e6a9952cf4238d34716f9a4749a5b880484c1faea2)
									
                                   图4 识别结果

![](https://ai-studio-static-online.cdn.bcebos.com/510134427ed949309271fe41b9916332b2ac3492a5e24c60aa8a9d2f826a8796)

					图6 专家咨询
											

# 专家系统
专家可以查看农户提出的疑问并进行解答，已解决的回复还可以修改后再次提交。如图6所示

![](https://ai-studio-static-online.cdn.bcebos.com/1e0d9242f1b943c68bf48e4936b04ec7e1a49801b33a4283993c9b38a344e5f1)


# 管理员系统
管理员输入个人账号进入到系统管理界面，负责对后台系统的维护工作，对农户和专家进行管理。还负责对专家申请的审核。如图7所示

![](https://ai-studio-static-online.cdn.bcebos.com/30c49d5705e34094be64f6b23c0ba7fa8d0b4ed563654c829178a5cb62353359)



请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 